In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# Global DataFrames (will be populated either from CSV or dummy data)
users_df = pd.DataFrame()
products_df = pd.DataFrame()
transactions_df = pd.DataFrame()
merged_df = pd.DataFrame()

def load_and_preprocess_data():
    """
    Loads data from CSV files or creates dummy dataframes if files are not found.
    Performs initial data cleaning and merging.
    Includes handling for new 'allergies' and 'allergens_present' columns.
    """
    global users_df, products_df, transactions_df, merged_df

    try:
        users_df = pd.read_csv('fake_users.csv')
        products_df = pd.read_csv('fake_products.csv')
        transactions_df = pd.read_csv('fake_transactions.csv')
        print("Data loaded successfully from CSVs.")
    except FileNotFoundError as e:
        print(f"Error loading file: {e}. Creating dummy dataframes for demonstration.")
        # Create dummy dataframes if files are not found
        # Dummy Users - Added 'allergies' column
        users_data = {
            'user_id': ['U001', 'U002', 'U003', 'U004', 'U005'],
            'diet_type': ['non-vegetarian', 'vegetarian', 'vegan', 'non-vegetarian', 'vegetarian'],
            'allergies': ['', 'nuts,gluten', 'dairy,nuts', '', 'gluten'] # Example allergies
        }
        users_df = pd.DataFrame(users_data)

        # Dummy Products - Added 'allergens_present' column
        products_data = {
            'product_id': ['P101', 'P102', 'P103', 'P104', 'P105', 'P106', 'P107', 'P108', 'P109', 'P110', 'P111', 'P112'],
            'product_name': ['Chicken Breast', 'Tofu', 'Milk', 'Eggs (Dozen)', 'Spinach',
                             'Ground Beef', 'Vegan Cheese', 'Yogurt', 'Salmon Fillet', 'Lentils',
                             'Almond Butter', 'Wheat Bread'],
            'category': ['Meat', 'Proteins', 'Dairy', 'Eggs', 'Vegetables',
                         'Meat', 'Dairy Alternative', 'Dairy', 'Seafood', 'Legumes',
                         'Spreads', 'Bakery'],
            'diet_type': ['non-vegetarian', 'vegetarian', 'dairy', 'eggs', 'vegetarian',
                                       'non-vegetarian', 'vegan', 'dairy', 'non-vegetarian', 'vegetarian',
                                       'vegan', 'vegetarian'],
            'expiry_date': [
                (datetime.now() + timedelta(days=5)).strftime('%Y-%m-%d'),
                (datetime.now() + timedelta(days=2)).strftime('%Y-%m-%d'),
                (datetime.now() + timedelta(days=8)).strftime('%Y-%m-%d'),
                (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d'), # Very close to expiration
                (datetime.now() + timedelta(days=10)).strftime('%Y-%m-%d'),
                (datetime.now() + timedelta(days=6)).strftime('%Y-%m-%d'),
                (datetime.now() + timedelta(days=4)).strftime('%Y-%m-%d'),
                (datetime.now() + timedelta(days=9)).strftime('%Y-%m-%d'),
                (datetime.now() + timedelta(days=3)).strftime('%Y-%m-%d'),
                (datetime.now() + timedelta(days=15)).strftime('%Y-%m-%d'),
                (datetime.now() + timedelta(days=20)).strftime('%Y-%m-%d'), # Almond Butter
                (datetime.now() + timedelta(days=4)).strftime('%Y-%m-%d'),  # Wheat Bread
            ],
            'allergens': [
                '', '', 'dairy', 'eggs', '',
                '', '', 'dairy', '', '',
                'nuts', 'gluten' # Example allergens
            ]
        }
        products_df = pd.DataFrame(products_data)

        # Dummy Transactions (some users buying some products)
        transactions_data = {
            'transaction_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
            'user_id': ['U001', 'U002', 'U001', 'U003', 'U002', 'U004', 'U001', 'U005', 'U003', 'U002', 'U001', 'U005'],
            'product_id': ['P101', 'P102', 'P104', 'P106', 'P101', 'P103', 'P105', 'P102', 'P107', 'P104', 'P111', 'P112'],
            'purchase_date': [
                (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d'),
                (datetime.now() - timedelta(days=25)).strftime('%Y-%m-%d'),
                (datetime.now() - timedelta(days=20)).strftime('%Y-%m-%d'),
                (datetime.now() - timedelta(days=15)).strftime('%Y-%m-%d'),
                (datetime.now() - timedelta(days=10)).strftime('%Y-%m-%d'),
                (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d'),
                (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d'),
                (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d'),
                (datetime.now() - timedelta(days=0)).strftime('%Y-%m-%d'),
                (datetime.now() - timedelta(days=0)).strftime('%Y-%m-%d'),
                (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d'), # Almond Butter
                (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d'), # Wheat Bread
            ],
            'quantity': [1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1],
            'price': [10.0, 5.0, 3.0, 4.0, 10.0, 12.0, 2.0, 6.0, 8.0, 4.0, 7.0, 3.5]
        }
        transactions_df = pd.DataFrame(transactions_data)
        print("Dummy dataframes created.")

    # --- Data Preprocessing ---
    # 1. Users DataFrame: Ensure diet_type and allergies are clean
    users_df['diet_type'] = users_df['diet_type'].str.lower().fillna('unknown')
    # Convert allergies string to a set of individual allergens for efficient lookup
    users_df['allergies'] = users_df['allergies'].apply(
        lambda x: set(item.strip() for item in x.lower().split(',')) if pd.notnull(x) and x.strip() != '' else set()
    )

    # 2. Products DataFrame:
    #    - Ensure 'expiry_date' is in datetime format.
    products_df['expiry_date'] = pd.to_datetime(products_df['expiry_date'], errors='coerce')
    # Check for NaT values that might cause issues and drop rows with them
    if products_df['expiry_date'].isnull().any():
        print("Warning: Some 'expiry_date' values could not be parsed and are NaT. Dropping these rows.")
        products_df.dropna(subset=['expiry_date'], inplace=True)

    products_df['diet_type'] = products_df['diet_type'].str.lower().fillna('unknown')
    # Convert allergens string to a set of individual allergens
    products_df['allergens'] = products_df['allergens'].apply(
        lambda x: set(item.strip() for item in x.lower().split(',')) if pd.notnull(x) and x.strip() != '' else set()
    )

    # 3. Transactions DataFrame:
    #    - Ensure 'purchase_date' is in datetime format.
    transactions_df['purchase_date'] = pd.to_datetime(transactions_df['purchase_date'])

    # Merge dataframes for easier access
    # Merge transactions with products to get product details for each transaction
    merged_df = pd.merge(transactions_df, products_df, on='product_id', how='left')
    # Merge the result with users to get user dietary preferences and allergies
    merged_df = pd.merge(merged_df, users_df, on='user_id', how='left')

    # Display basic info and head of merged_df to verify
    print("\nMerged DataFrame Info:")
    merged_df.info()
    print("\nMerged DataFrame Head:")
    print(merged_df.head())

def get_recommendations(user_id, current_date=None, expiration_threshold_days=20, max_discount=0.5):
    """
    Generates product recommendations for a given user, prioritizing products
    close to their expiration date with increasing discounts, while respecting
    dietary preferences and avoiding allergens.

    Args:
        user_id (int): The ID of the user for whom to generate recommendations.
        current_date (datetime.date, optional): The current date to calculate
                                                days until expiration. Defaults to today.
        expiration_threshold_days (int): Products expiring within this many days
                                         will be considered for recommendation.
                                         (Now set to 20 days by default)
        max_discount (float): Maximum discount percentage (e.g., 0.5 for 50%).

    Returns:
        list: A list of dictionaries, each containing recommended product details
              (product_id, product_name, days_until_expiration, recommended_discount).
    """
    if current_date is None:
        current_date = datetime.now().date()

    user_info = users_df[users_df['user_id'] == user_id]
    if user_info.empty:
        print(f"User ID {user_id} not found.")
        return []

    user_diet_type = user_info['diet_type'].iloc[0]
    user_allergies = user_info['allergies'].iloc[0] # Get the set of user allergies
    user_purchased_products = merged_df[merged_df['user_id'] == user_id]['product_id'].unique()

    # 1. Identify products close to expiration (between 3 and expiration_threshold_days)
    products_df['days_until_expiration'] = products_df['expiry_date'].apply(
        lambda x: (x.date() - current_date).days if pd.notnull(x) else np.nan
    )
    expiring_products = products_df[
        (products_df['days_until_expiration'] >= 3) & # Must be at least 3 days from expiry
        (products_df['days_until_expiration'] <= expiration_threshold_days)
    ].copy()

    if expiring_products.empty:
        print(f"No products expiring within the {3}-{expiration_threshold_days} day window.")
        return []

    # 2. Apply dietary filters
    if user_diet_type == 'vegetarian':
        # Exclude meat and eggs
        expiring_products = expiring_products[
            (expiring_products['diet_type'] != 'non-vegetarian') &
            (expiring_products['diet_type'] != 'eggs')
        ]
    elif user_diet_type == 'vegan':
        # Exclude meat, eggs, and dairy
        expiring_products = expiring_products[
            (expiring_products['diet_type'] != 'non-vegetarian') &
            (expiring_products['diet_type'] != 'eggs') &
            (expiring_products['diet_type'] != 'dairy')
        ]
    # For 'non-vegetarian' or 'unknown', no specific exclusions based on diet

    # 3. Apply allergy filters
    if user_allergies: # Only filter if the user has specified allergies
        # Filter out products where any of the product's allergens are in the user's allergies
        expiring_products = expiring_products[
            ~expiring_products['allergens'].apply(
                lambda product_allergens: bool(user_allergies.intersection(product_allergens))
            )
        ]

    if expiring_products.empty:
        print(f"No suitable expiring products after applying dietary and allergy filters for user {user_id}.")
        return []

    # 4. Calculate gradually increasing discount
    # Discount increases linearly as days_until_expiration decreases
    # The discount calculation now considers the new 3-day minimum
    # If a product expires in 3 days, it gets the max discount. If it expires in 20 days, it gets 0% (or very low).
    # Formula: discount = max_discount * (1 - (days_until_expiration - min_days_for_discount) / (expiration_threshold_days - min_days_for_discount))
    min_days_for_discount = 3
    if expiration_threshold_days > min_days_for_discount:
        expiring_products['recommended_discount'] = expiring_products['days_until_expiration'].apply(
            lambda x: max_discount * (1 - (x - min_days_for_discount) / (expiration_threshold_days - min_days_for_discount))
            if x >= min_days_for_discount else 0
        )
    else: # If threshold is 3 days or less, then max_discount is applied on day 3
        expiring_products['recommended_discount'] = expiring_products['days_until_expiration'].apply(
            lambda x: max_discount if x == min_days_for_discount else 0
        )

    expiring_products['recommended_discount'] = expiring_products['recommended_discount'].clip(lower=0, upper=max_discount)

    # 5. Prioritize based on purchase history
    expiring_products['previously_purchased'] = expiring_products['product_id'].isin(user_purchased_products)
    expiring_products = expiring_products.sort_values(
        by=['previously_purchased', 'days_until_expiration'],
        ascending=[False, True]
    )

    recommendations = expiring_products[[
        'product_id', 'name', 'days_until_expiration', 'recommended_discount'
    ]].to_dict(orient='records')

    return recommendations

# --- Main execution ---
if __name__ == "__main__":
    load_and_preprocess_data()

    print("\n--- Example Recommendations ---")

    # Example 1: User 1 (non-vegetarian, no allergies)
    user_id_1 = 'U0085' # Using a user ID from the merged_df head
    print(f"\nRecommendations for User {user_id_1} (Non-Vegetarian, No Allergies):")
    recs_1 = get_recommendations(user_id_1)
    if recs_1:
        for rec in recs_1[:10]: # Cap to 10 recommendations
            print(f"  Product: {rec['name']} (ID: {rec['product_id']}), "
                  f"Expires in: {rec['days_until_expiration']} days, "
                  f"Discount: {rec['recommended_discount']:.2%}")
    else:
        print("  No recommendations found.")

    # Example 2: User 2 (vegetarian, nuts & gluten allergies)
    user_id_2 = 'U0083' # Using a user ID from the merged_df head
    print(f"\nRecommendations for User {user_id_2} (Vegetarian, Nuts & Gluten Allergies):")
    recs_2 = get_recommendations(user_id_2)
    if recs_2:
        for rec in recs_2[:10]: # Cap to 10 recommendations
            print(f"  Product: {rec['name']} (ID: {rec['product_id']}), "
                  f"Expires in: {rec['days_until_expiration']} days, "
                  f"Discount: {rec['recommended_discount']:.2%}")
    else:
        print("  No recommendations found.")

    # Example 3: User 3 (vegan, dairy & nuts allergies)
    user_id_3 = 'U0138' # Using a user ID from the merged_df head
    print(f"\nRecommendations for User {user_id_3} (Vegan, Dairy & Nuts Allergies):")
    recs_3 = get_recommendations(user_id_3)
    if recs_3:
        for rec in recs_3[:10]: # Cap to 10 recommendations
            print(f"  Product: {rec['name']} (ID: {rec['product_id']}), "
                  f"Expires in: {rec['days_until_expiration']} days, "
                  f"Discount: {rec['recommended_discount']:.2%}")
    else:
        print("  No recommendations found.")

    # Example 4: User 5 (vegetarian, gluten allergy)
    user_id_5 = 'U0144' # Using a user ID from the merged_df head
    print(f"\nRecommendations for User {user_id_5} (Vegetarian, Gluten Allergy):")
    recs_5 = get_recommendations(user_id_5)
    if recs_5:
        for rec in recs_5[:10]: # Cap to 10 recommendations
            print(f"  Product: {rec['name']} (ID: {rec['product_id']}), "
                  f"Expires in: {rec['days_until_expiration']} days, "
                  f"Discount: {rec['recommended_discount']:.2%}")
    else:
        print("  No recommendations found.")


    # Example 5: User with no expiring products or all filtered out (using a non-existent user ID)
    user_id_non_existent = 'U999'
    print(f"\nRecommendations for User {user_id_non_existent} (Non-existent or no suitable products):")
    recs_non_existent = get_recommendations(user_id_non_existent)
    if recs_non_existent:
        for rec in recs_non_existent[:10]: # Cap to 10 recommendations
            print(f"  Product: {rec['name']} (ID: {rec['product_id']}), "
                  f"Expires in: {rec['days_until_expiration']} days, "
                  f"Discount: {rec['recommended_discount']:.2%}")
    else:
        print("  No recommendations found.")

Data loaded successfully from CSVs.

Merged DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   user_id                     1000 non-null   object        
 1   product_id                  1000 non-null   object        
 2   purchase_date               1000 non-null   datetime64[ns]
 3   quantity                    1000 non-null   int64         
 4   price_paid_per_unit         1000 non-null   float64       
 5   total_price_paid            1000 non-null   float64       
 6   discount_percent            1000 non-null   int64         
 7   product_diet_type           1000 non-null   object        
 8   user_diet_type              1000 non-null   object        
 9   days_to_expiry_at_purchase  1000 non-null   int64         
 10  user_engaged_with_deal      1000 non-null   int64         
 1

# Task
Calculate seller savings from selling expiring products at a discount and user savings for each user based on purchasing the top 2 discounted recommendations, and output these savings along with the top 10 recommendations per user.

## Calculate seller savings

### Subtask:
Determine the total potential loss from all expiring products if they were not sold, and then calculate the savings achieved by selling them at a discount.


**Reasoning**:
Filter products expiring within the threshold and calculate the total potential loss and revenue from discounted sales to determine seller savings.



In [ ]:
# 1. Filter products expiring within the threshold (using expiration_threshold_days from get_recommendations function)
# Note: Assuming expiration_threshold_days and min_days_for_discount are the same as in get_recommendations
expiration_threshold_days = 20
min_days_for_discount = 3

expiring_products_for_savings = products_df[
    (products_df['days_until_expiration'] >= min_days_for_discount) &
    (products_df['days_until_expiration'] <= expiration_threshold_days)
].copy()

# Ensure recommended_discount is calculated for these products (same logic as in get_recommendations)
if expiration_threshold_days > min_days_for_discount:
    expiring_products_for_savings['recommended_discount'] = expiring_products_for_savings['days_until_expiration'].apply(
        lambda x: 0.5 * (1 - (x - min_days_for_discount) / (expiration_threshold_days - min_days_for_discount))
        if x >= min_days_for_discount else 0
    )
else:
    expiring_products_for_savings['recommended_discount'] = expiring_products_for_savings['days_until_expiration'].apply(
        lambda x: 0.5 if x == min_days_for_discount else 0
    )
expiring_products_for_savings['recommended_discount'] = expiring_products_for_savings['recommended_discount'].clip(lower=0, upper=0.5)


# 2. Calculate the total potential loss (assuming 1 unit per product)
total_potential_loss = expiring_products_for_savings['price_mrp'].sum()

# 3. Calculate the revenue from selling these expiring products at a discount
expiring_products_for_savings['discounted_price'] = expiring_products_for_savings['price_mrp'] * (1 - expiring_products_for_savings['recommended_discount'])
total_discounted_revenue = expiring_products_for_savings['discounted_price'].sum()

# 4. Calculate the seller savings
seller_savings = total_potential_loss - total_discounted_revenue

print(f"Total Potential Loss from expiring products: ${total_potential_loss:.2f}")
print(f"Total Revenue from selling at discount: ${total_discounted_revenue:.2f}")
print(f"Seller Savings from discounted sales: ${seller_savings:.2f}")

Total Potential Loss from expiring products: $7165.68
Total Revenue from selling at discount: $5405.17
Seller Savings from discounted sales: $1760.51


## Calculate user savings

### Subtask:
For each user, identify the top 2 recommended products with the highest discounts and calculate the total savings for the user if they purchase these products.


**Reasoning**:
Iterate through the specified user IDs, get recommendations, sort by discount, select the top 2, calculate savings based on original price, and store the user ID and total savings.



In [ ]:
user_ids_for_savings = ['U0085', 'U0083', 'U0138', 'U0144']
user_savings = {}

for user_id in user_ids_for_savings:
    recommendations = get_recommendations(user_id)
    if recommendations:
        # Sort recommendations by recommended_discount in descending order
        sorted_recommendations = sorted(recommendations, key=lambda x: x['recommended_discount'], reverse=True)

        # Select the top 2 recommendations
        top_2_recommendations = sorted_recommendations[:2]

        total_saving_for_user = 0
        for rec in top_2_recommendations:
            # Look up the original price (price_mrp) from products_df
            product_id = rec['product_id']
            product_info = products_df[products_df['product_id'] == product_id]

            if not product_info.empty:
                original_price = product_info['price_mrp'].iloc[0]
                saving = original_price * rec['recommended_discount']
                total_saving_for_user += saving
            else:
                print(f"Warning: Product ID {product_id} not found in products_df.")

        user_savings[user_id] = total_saving_for_user

# Print the user savings
print("\n--- User Savings from Top 2 Discounted Recommendations ---")
for user_id, savings in user_savings.items():
    print(f"User {user_id}: ${savings:.2f}")


--- User Savings from Top 2 Discounted Recommendations ---
User U0085: $206.83
User U0083: $206.83
User U0138: $202.59
User U0144: $206.83


## Output results

### Subtask:
Print the calculated seller savings and the user savings for each example user.


**Reasoning**:
Print the seller savings and iterate through the user savings dictionary to print each user's savings.



In [ ]:
# Print the calculated seller savings
print("\n--- Seller Savings ---")
print(f"Total Potential Loss from expiring products: ${total_potential_loss:.2f}")
print(f"Total Revenue from selling at discount: ${total_discounted_revenue:.2f}")
print(f"Seller Savings from discounted sales: ${seller_savings:.2f}")

# Print the user savings
print("\n--- User Savings from Top 2 Discounted Recommendations ---")
for user_id, savings in user_savings.items():
    print(f"User {user_id}: ${savings:.2f}")


--- Seller Savings ---
Total Potential Loss from expiring products: $7165.68
Total Revenue from selling at discount: $5405.17
Seller Savings from discounted sales: $1760.51

--- User Savings from Top 2 Discounted Recommendations ---
User U0085: $206.83
User U0083: $206.83
User U0138: $202.59
User U0144: $206.83


## Summary:

### Data Analysis Key Findings

*   The total potential loss from expiring products was calculated to be \$7165.68.
*   Selling these expiring products at a discount generated a total revenue of \$5405.17.
*   The seller savings from these discounted sales amounted to \$1760.51.
*   User savings from purchasing the top 2 discounted recommendations were calculated for four example users: U0085, U0083, U0138, and U0144.
*   The user savings ranged from \$202.59 to \$206.83 for these users.

### Insights or Next Steps

*   The discount strategy for expiring products is effective in recovering a significant portion of potential losses, resulting in substantial seller savings.
*   Highlighting the calculated user savings could be a compelling way to promote discounted expiring products and encourage purchases.
